### 데이터셋 결합하기 : 병합과 조인

Pandas는 pd.merge() 함수를 활용하여 고성능 인메모리 조인과 병합연산을 할 수 있다

#### 조인 작업의 분류

pd.merge() 함수는 일대일, 다대일, 다대다 같은 여러가지 조인 유형을 구현한다.

In [197]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'employee':['Bob', 'Jake','Lisa','Sue'],
                   'group': ['Accounting', 'Engineering', 
                             'Engineering', 'HR']})

In [198]:
df2 = pd.DataFrame({'employee' : ['Lisa','Bob','Jake','Sue'], 
                    'hire_date': [2004, 2000, 2012, 2014]})

In [199]:
print(df1) ; print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2000
2     Jake       2012
3      Sue       2014


In [200]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2000
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 다대일(Many-to-one) 조인

병합하는 키 열 하나에 중복된 항목이 포함되는 경우의 조인

In [201]:
df4 = pd.DataFrame({'group':['Accounting','Engineering', 'HR'],
                             'supervisor':['Carly', 'Guido','Steve']})

In [202]:
print(df4) ; print(df3)

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date
0      Bob   Accounting       2000
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


In [203]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2000,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [204]:
### 다대다(Many-to-many) 조인

병합되는 두 데이터프레임에서 키열에 대해 모두 중복항목이 존재하면 다대다 조인이다.

In [205]:
df5 = pd.DataFrame({'group':
                    ['Accounting','Accounting',
                     'Engineering','Engineering','HR','HR'],
                   'skills':
                    ['math','spreadsheets','coding',
                     'linux','spreadsheets','organiation']})

In [206]:
print(df1) ; print(df5)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR   organiation


In [207]:
pd.merge(df1,df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organiation


#### 병합 키 지정

pd.merge()는 두개의 입력 데이터셋 사이에 일치하는 하나 이상의 열 이름을 찾아 그것을 키로 사용한다.

#### on 키워드

on 키워드를 사용해 키로 쓸 열이름을 명시적으로 지정할 수 있다.

In [208]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [209]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2000
2,Jake,2012
3,Sue,2014


In [210]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2000
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### left_on과 right_on 키워드

다른 열이름을 가진 두 데이터셋을 병합하려면, left_on과 right_on 키워드를 활용한다

In [211]:
df3 = pd.DataFrame({'name':['Bob','Jake','Lisa','Sue'],
                   'salary':[70000,80000,120000,90000]})

In [212]:
print(df1); print(df3)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000


In [213]:
pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [214]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### left_index와 right_index 키워드 그리고 join() 메서드

기본적으론 열을 기준으로 병합하지만 left_index, right_index를 활용하면 인덱스로 병합할 수 있다.

In [215]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [216]:
df1a = df1.set_index('employee')

In [217]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [218]:
df2a = df2.set_index('employee')

In [219]:
df2a

,hire_date
employee,
Lisa,2004
Bob,2000
Jake,2012
Sue,2014


In [220]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2000
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [221]:
pd.merge(df1a, df2a, left_index=True, right_index=True).reset_index()

,employee,group,hire_date
0,Bob,Accounting,2000
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [222]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2000
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


### 인덱스와 열

두 데이터셋에서 하나는 인덱스로 다른 것은 열로 섞고자 하면 left_index와 right_on, left_on과 right_index를 조합할 수 있다.

In [223]:
print(df1a) ; print(df3)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000


In [224]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


In [225]:
print(df1) ; print(df1a)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR


##### Quiz) 두 개(df1, df1a)의 데이터 프레임을 병합해 주세요

In [226]:
pd.merge(df1, df1a, left_on='employee', right_index=True)

,employee,group_x,group_y
0,Bob,Accounting,Accounting
1,Jake,Engineering,Engineering
2,Lisa,Engineering,Engineering
3,Sue,HR,HR


### 조인을 위한 집합연산 지정하기

데이터셋에 따라 키열에 맞는 값이 등장하거나 등장하지 않는 경우가 있다.

In [227]:
df6 = pd.DataFrame({'name':['Peter','Paul','Mary'],
                   'food':['fish','beans','bread']},
                  columns=['name','food'])

In [228]:
df7 = pd.DataFrame({'name':['Mary','Joseph'],
                   'drink':['wine','beer']},
                  columns=['name','drink'])

In [229]:
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


결과가 두 데이터 집합에 대한 교집합이 들어간다. 이를 내부조인(inner join)이라 한다. how 키워드를 활용하여 outer, left, right로 기준을 정할 수 있다.

In [230]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [231]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [232]:
print(df6) ; print(df7)

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer


In [233]:
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


#### 열 이름이 겹치는 경우 : suffixes 키워드

두 데이터 셋에서 열이름이 충돌하는 경우 자동으로 접미사 _x, _y가 붙는다.

In [234]:
df8 = pd.DataFrame({'name':['Bob','Jake','Lisa','Sue'],'rank':[1,2,3,4]})

In [235]:
df9 = pd.DataFrame({'name':['Bob','Jake','Lisa','Sue'],'rank':[3,1,4,2]})

In [236]:
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [237]:
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [238]:
pd.merge(df8, df9, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


기본값이 싫다면 suffixes 키워드를 사용해 별도로 지정할 수 있다.

In [239]:
pd.merge(df8, df9, on="name", 
         suffixes=["_left","_right"])

,name,rank_left,rank_right
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


##### Quiz
두개의 데이터 셋(state-abbresv.csv, state-population.csv)을 외부조인 하여 병합하세요

In [240]:
pop = pd.read_csv('state-population.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [241]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [242]:
pd.merge(abbrevs, pop, left_on='abbreviation', 
         right_on='state/region', how='outer').drop('state/region',
                                                   axis=1)

,state,abbreviation,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0
5,Alabama,AL,total,2011,4801627.0
6,Alabama,AL,total,2009,4757938.0
7,Alabama,AL,under18,2009,1134192.0
8,Alabama,AL,under18,2013,1111481.0
9,Alabama,AL,total,2013,4833722.0


In [243]:
pop.describe()

,year,population
count,2544.000000,2.524000e+03
mean,2001.500000,6.805558e+06
std,6.923547,2.855014e+07
min,1990.000000,1.013090e+05
25%,1995.750000,7.423805e+05
50%,2001.500000,1.597005e+06
75%,2007.250000,4.547104e+06
max,2013.000000,3.161288e+08


## GroupBy: 분할, 적용, 결합

각 열에 대한 여러 일반적인 집계를 계산하고 결과를 반환하는 describe() 메서드가 있다. 간단한 집계는 데이터세트의 전반적인 특성을 알려주지만, 때에 따라서는 어떤 행이나 인덱스를 기준으로 조건부로 집계할 경우가 있다. 이때 GroupBy 연산을 활용한다.

#### 분할, 적용, 결합

- 분할 : 지정된 키값을 기준으로 데이터프레임을 나누고 분류
- 적용 : 개별 그룹 내에서 일반적으로 집계, 변환, 필터링 같은 함수를 계산
- 결합 : 연산의 결과를 결과 배열에 병합

In [244]:
df = pd.DataFrame({'key' : ['A','B','C','A','B','C'],
                  'data' : range(6)},
                 columns = ['key','data'])

In [193]:
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


##### Tip) 컬럼 순서 바꾸기

In [245]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [246]:
#데이터프레임[컬럼리스트]
pop = pop[['population','year','ages','state/region']]

In [247]:
pop.head()

,population,year,ages,state/region
0,1117489.0,2012,under18,AL
1,4817528.0,2012,total,AL
2,1130966.0,2010,under18,AL
3,4785570.0,2010,total,AL
4,1125763.0,2011,under18,AL


데이터프레임의 groupby()메서드에 원하는 키 열의 이름을 전달해 가장 기본적인 분할, 적용, 결합 연결을 계산

In [248]:
df.groupby('key')

In [249]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [250]:
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


### 집계, 필터, 변환, 적용

sum(), median() 등을 활용하여 데이터프레임의 데이터를 집계하였지만 aggregate() 메서드를 활용하면 더 유연하게 집계 할 수 있다.

In [251]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key' : ['A','B','C','A','B','C'],
                  'data1' : range(6),
                  'data2' : rng.randint(0,10,6)},
                 columns = ['key','data1','data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [252]:
df.groupby('key').aggregate([min, max, np.median])

data1            data2           
      min max median   min max median
key                                  
A       0   3    1.5     3   5    4.0
B       1   4    2.5     0   7    3.5
C       2   5    3.5     3   9    6.0

열 이름을 해당 열에 적용될 연산에 매핑하는 딕셔너리를 전달하여 같은 동작을 수행할 수 있다.

In [255]:
df.groupby('key').aggregate({'data1':'min', 'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [256]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


### 필터링


필터링 연산을 사용하면 그룹 속성을 기준으로 데이터를 걸러낼 수 있다.

In [257]:
def filter_func(x):
    return x['data2'].std() > 4

In [258]:
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [259]:
df.groupby('key').filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


#### 변환

데이터의 내용을 재 계산하여 데이터가 바뀐 버전의 새로운 데이터프레임을 만들 수 있다.

In [261]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [265]:
df.groupby('key').mean()

,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


In [266]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


함수를 활용하고 싶다면 apply() 메서드를 활용한다.

In [269]:
def norm_by_data2(x):
    x['data13'] = x['data2'].sum()
    return x

In [270]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2,data13
0,A,0,5,8
1,B,1,0,7
2,C,2,3,12
3,A,3,3,8
4,B,4,7,7
5,C,5,9,12


##### quiz) 
위의 예제에서 groupby를 활용하여 key를 기준으로한 data2의 평균을 표시하세요.

In [271]:
def norm_by_data2(x):
    x['평균'] = x['data2'].mean()
    return x

In [273]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2,평균
0,A,0,5,4.0
1,B,1,0,3.5
2,C,2,3,6.0
3,A,3,3,4.0
4,B,4,7,3.5
5,C,5,9,6.0


### 분할 키 지정하기(그룹 지정)

하나의 열 이름을 기준으로 DataFrame을 분할 하였지만 리스트, 딕셔너리, 리스트를 활용하여 그룹을 만들수 있다.

In [275]:
L = [0,1,0,1,2,0]
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [276]:
df.groupby(L).sum()# L = [0,1,0,1,2,0]

,data1,data2
0,7,17
1,4,3
2,4,7


### 딕셔너리

In [280]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [282]:
df2 = df.set_index('key')
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [283]:
mapping = {'A':'Banana','B':'Apple','C':'Apple'}

In [284]:
df2.groupby(mapping).sum()

,data1,data2
Apple,12,19
Banana,3,8


#### 인덱스

인덱스의 그룹을 출력하는 파이썬함수를 활용할 수 있다.

In [287]:
df2.index

Index(['A', 'B', 'C', 'A', 'B', 'C'], dtype='object', name='key')

In [286]:
df2.groupby(df2.index).mean()

,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


앞에서 다룬 방식을 리스트로 다중인덱스로 구성할 수 있다.

In [289]:

df2.groupby([df2.index, mapping]).mean()

,,data1,data2
key,,,
A,Banana,1.5,4.0
B,Apple,2.5,3.5
C,Apple,3.5,6.0


#### 예시

In [293]:
import seaborn as sns
planets = sns.load_dataset('planets')

In [294]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [301]:
decade = (planets['year'] // 10) * 10
decade = decade.astype(str)+'s'
decade.head()

0    2000s
1    2000s
2    2010s
3    2000s
4    2000s
Name: year, dtype: object

In [306]:
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

year,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
